In [ ]:
from sklearn.naive_bayes import MultinomialNB  # 朴素贝叶斯 API
import pandas as pd
import codecs  #用来加载GBK编码文件  codecs
import re    # 正则
import jieba   # 中文分词库
import time
from sklearn.feature_extraction.text import CountVectorizer # 文本计数向量表示
import numpy as np
from sklearn.model_selection import train_test_split
import joblib
from collections import Counter
import random

filename1 = 'trec06c/data1.csv'
filename2 = 'trec06c/data2.csv'
filename3 = 'trec06c/data3.csv'

## 读取邮件数据

In [5]:
email_labels = []  # 用来保存标签
email_contents = []  # 用来保存邮件的内容
sampe_number = 1000
# 读取所有邮件
for line in open('trec06c/full/index',errors='ignore'):  #  errors = 'ignore' 忽略错误
    # spam 垃圾邮件   ham 非垃圾邮件
    label ,data = line.strip().split() # spam ../data/000/000  利用空格拆分
    #读取邮件内容
    file_name = 'trec06c'+data[2:]  # data = '../data/000/000'  跳过..
    file_data = codecs.open(file_name,'r','gbk',errors='ignore').read() # 这里指定编码为GBK

    email_labels.append(label)
    email_contents.append(file_data)

email_data = pd.DataFrame({'content':email_contents,'label':email_labels}) # 通过dataframe 来保存结果
spam_email = email_data[email_data['label']=='spam'].sample(sampe_number) #从所有垃圾邮件中采样1000封
ham_email = email_data[email_data['label']=='ham'].sample(sampe_number) # 从所欲非垃圾邮件中采样1000封

email_data = pd.concat([spam_email,ham_email]) # concat 默认上下拼接
email_data.to_csv(filename1)

NameError: name 'filename1' is not defined

## 邮件数据处理

In [10]:
# 加载上一步处理好的文件
email_data = pd.read_csv(filename1)
contents= []
for index,email in enumerate(email_data['content'],1):
    # 去除换行符
    email = email.replace('\n',' ')
    # 去除非中文内容
    email = re.sub('[^\u4e00-\u9fff]', '', email)
    # 去除多余空白
    email = ' '.join(email.split())
    # 中文分词
    email = ' '.join(jieba.lcut(email)) # 使用jieba 来进行中文分词
    # 保存分词的结果
    contents.append(email)
data = pd.DataFrame({'content':contents,'label':email_data['label']})
data.to_csv(filename2)

NameError: name 'filename1' is not defined

In [7]:
text = '每一位管理和技术人员都清楚地懂得，单纯从技术角度衡量为合算的方案'
jieba.lcut(text)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.768 seconds.
Prefix dict has been built successfully.


['每',
 '一位',
 '管理',
 '和',
 '技术人员',
 '都',
 '清楚',
 '地',
 '懂得',
 '，',
 '单纯',
 '从',
 '技术',
 '角度',
 '衡量',
 '为',
 '合算',
 '的',
 '方案']

In [25]:
# 读取上一步处理好的文件
email = pd.read_csv(filename2)
email.dropna(inplace=True) # 处理缺失值
stop_words = [] #停用词表
for word in open('data/trec06c/stoplist.txt','r',encoding = 'gbk'):
    stop_words.append(word.strip())
# 创建 CountVectorizer   用词频来做向量表示
transformer = CountVectorizer(stop_words=stop_words) # 传入停用词表
# CountVectorizer 处理数据 不能包含缺失值
x = transformer.fit_transform(email['content']).toarray() #特征
y = np.where(email['label'].values =='ham',0,1)  # 目标
data= pd.DataFrame(x)
data[x.shape[1]] = y   #  数据中一共有x.shape[1] 列, 最后一列用来保存标签
data.to_csv(filename3)

In [36]:
transformer = CountVectorizer(stop_words=stop_words) # 1944篇文章  统计出现多少个不同的词 ,每一个单词相当于是一个特征维度  每一个样本, 在对应维度上 出现了相关单词 就记录对应次数
transformer.fit_transform(email['content']).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [39]:
transformer.get_feature_names()[5]

'一万五千'

In [46]:
email['content'][16]

'纺织 行业 专用 管理软件 互联网 版 提供 免费 在线 试用 联系电话 联系人 赵纠 您 是否 遇到 过 以下 的 问题 不 知道 现在 准确 的 库存 布匹 数量 导致 错过 交货期 每 一匹 布 长度 缸 号 规格 等级 不同 仓库 管理 销售 管理 困难 样板 寄送 费用 高 或 对 相同 客户 重复 寄送 导致 损失 业务 跟单 工作 繁忙 查找 历史 报价 需要 大量 时间 不能 有效 的 施行 库存 预留 导致 的 货卖 给 了 不 知道 采购 未到 仓 数量 导致 砉 有效 订单 不能 及时 计算 出 准确 的 仓租 到期日 导致 公司 客户 造成 损失 分公司 管理 困难 了解 工厂 或 其他 分公司 的 数据 有 延迟 不能 实时 查看 到 往 拿到 一份 报表 需要 很 长时间 找 不到 真正 适合 纺织 行业 管理 的 软件 找 不到 繁简通 用 的 软件 繁体 系统 看繁 澹 简体 系统 看 简体 只能 勉强 使用 其他软件 不能 做到 对 资料 按 权限 进行 隔离 使 个人 只 接触 到 他 需要 的 资料 避免 商业 机芡 庑 埂 成功 案例 之一 三亿 纺织 有限公司 是 一家 专业 的 纺织 用品 代 砩 蹋 包括 近 万种 花色 布匹 有 六大类 近个 小 类产品 有 全国 总代理 产品 也 星域 代理 产品 有 产品 信息 一万五千 条 以前 有 业务 记录本 几 十多本 另外 客柿 瞎芾 砗 脱品 寄送 也 消耗 了 企业 很多 时间 老板 经常 为 管理 问题 而 发愁 现在 好 了 有 了 天朗 软件 这个 帮手 业务员 的 管理 产品 的 管理 销售 机会 的 管理 财务管理 一目了然 还有 好多 想不到 的 好处 了 不信 您 也 试用 一下 软件 大 优势 专业 的 纺织 行业 管理软件 解决 更 多 实际 问题 操作 简单 软件 稳定 为 使用者 节约 更 多 时间 一机 安装 各地 可用 性能 价格比 更高 本地化 服务 和 操作 培训 分享 更 多 管理 狙椤 采用 四层 安全 体系 企业 的 资料 数据 更 安全 新 太 天朗 企业 网络管理 软件 专业 为 纺织 行业 设计 的 互联网 管理软件 能够 帮 您 实现 实时 库存 管理 实时 订单 管理 实时 的 外 加工 管理 订单 管理 通过 订单 系

## 使用朴素贝叶斯分类

In [47]:
data = pd.read_csv(filename3)
x = data.iloc[:,:-1]
y = data.iloc[:,-1]

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=0)
# 创建朴素贝叶斯分类器对象
estimator = MultinomialNB(alpha=1.0) # alpha 就是拉普拉斯平滑系数 如果传入0 不考虑平滑问题 默认值是1
# 训练模型
estimator.fit(x_train,y_train)
 # 预测结果
estimator.score(x_test,y_test)

0.8586118251928021